In [4]:
import pandas as pd

df = pd.read_csv('../data/testresults.csv')

df[['test','accuracy0','accuracy1']]

,test,accuracy0,accuracy1
0,sympsblingfire,0.616822,0.703704
1,sympsnltk,0.635514,0.666667
2,sympsspacysm,0.672897,0.629630
3,sympsspacylg,0.644860,0.666667
4,sympsspacytrf,0.682243,0.722222
5,w2vec,0.887850,0.944444
6,sympsreweightedblingfire,0.607477,0.703704
7,sympsreweightednltk,0.542056,0.722222
8,sympsreweightedspacysm,0.700935,0.629630
9,sympsreweightedspacylg,0.672897,0.666667
